In [3]:
import torch 
import torch.nn as nn
import torchvision 
import torchvision.transforms as trsansforms
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
input_size = 784 
epochs = 2
hidden_size = 500
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [8]:
#MNIST

train_dataset = torchvision.datasets.MNIST(root = './data',
                                        train = True,
                                        transform=trsansforms.ToTensor(),
                                        download = True)

test_dataset = torchvision.datasets.MNIST(root = './data',
                                        train = False,
                                        transform=trsansforms.ToTensor(),
                                        download = True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

In [23]:
## Model definition
class Neuralnet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Neuralnet, self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size)
        self.Relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.Relu(x)
        x = self.layer2(x)

        return x

model = Neuralnet(input_size, hidden_size, num_classes).to(device)
print(model)

Neuralnet(
  (layer1): Linear(in_features=784, out_features=500, bias=True)
  (Relu): ReLU()
  (layer2): Linear(in_features=500, out_features=10, bias=True)
)


In [24]:
# Define model parameters

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [25]:
## Model training

n_total_steps = len(train_loader)

for epoch in range(epochs):
    for i, (image, label) in enumerate(train_loader):
        images = image.reshape(-1, 28*28).to(device)
        labels = label.to(device)

        optimizer.zero_grad()
        pred_labels = model(images)
        l = criterion(pred_labels, labels)

        l.backward()
        optimizer.step()

        if epoch%1==0:
            print(f"Epoch {epoch+1}/{epochs} , Step {i+1}/{n_total_steps}, Loss : {l.item():.4f}")


Epoch 1/2 , Step 1/600, Loss : 2.3101
Epoch 1/2 , Step 2/600, Loss : 2.2265
Epoch 1/2 , Step 3/600, Loss : 2.1364
Epoch 1/2 , Step 4/600, Loss : 2.0386
Epoch 1/2 , Step 5/600, Loss : 1.8903
Epoch 1/2 , Step 6/600, Loss : 1.8770
Epoch 1/2 , Step 7/600, Loss : 1.8283
Epoch 1/2 , Step 8/600, Loss : 1.6356
Epoch 1/2 , Step 9/600, Loss : 1.5979
Epoch 1/2 , Step 10/600, Loss : 1.4006
Epoch 1/2 , Step 11/600, Loss : 1.3493
Epoch 1/2 , Step 12/600, Loss : 1.2594
Epoch 1/2 , Step 13/600, Loss : 1.1910
Epoch 1/2 , Step 14/600, Loss : 1.2110
Epoch 1/2 , Step 15/600, Loss : 1.1386
Epoch 1/2 , Step 16/600, Loss : 1.0107
Epoch 1/2 , Step 17/600, Loss : 1.0028
Epoch 1/2 , Step 18/600, Loss : 0.8886
Epoch 1/2 , Step 19/600, Loss : 0.8525
Epoch 1/2 , Step 20/600, Loss : 0.8353
Epoch 1/2 , Step 21/600, Loss : 0.7662
Epoch 1/2 , Step 22/600, Loss : 0.7378
Epoch 1/2 , Step 23/600, Loss : 0.6958
Epoch 1/2 , Step 24/600, Loss : 0.6170
Epoch 1/2 , Step 25/600, Loss : 0.6807
Epoch 1/2 , Step 26/600, Loss : 0.

In [33]:
## Predictions 

with torch.no_grad():
    n_correct =0
    n_test_samples = len(test_loader.dataset)

    for i, (image, label) in enumerate(test_loader):
        images = image.reshape(-1,28*28).to(device)
        labels = label.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs,1)

        n_correct+= (predicted == labels).sum().item()

    acc = n_correct/n_test_samples

    print(f"Accuracy of the network with {n_test_samples} is {acc*100:.2f}")



Accuracy of the network with 10000 is 97.25
